# Pytorch에서 ResNet 구현하기
원문 링크: https://cryptosalamander.tistory.com/156?category=1181429

## 논문 제시 Architecture
최초 컨볼루션 이후 4개의 블럭으로 구성
BottelNeck Architecture

In [2]:
import torch 

# 파이토치 레이어 관련
import torch.nn as nn

# Activation func 관련
import torch.nn.functional as F

## Basic Residual Block (ResNet 18, 34)

In [5]:
class BasicBlock(nn.Module):
    # mul -> ResNet 18, 34, 50, 101, 152 구조 생성에 사용됨
    mul = 1
    def __init__(self, in_planes, out_planes, stride=1):
        super(BasicBlock, self).__init__()
        
        # stride로 이미지 너비와 높이 조정
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        
        # 넓이, 높이는 그대로 유지
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        
        # 입력 x 르를 그대로 더해줌
        self.shortcut = nn.Sequential()
        
        # 만약 size가 안맞아 합연산이 불가하면, 연산이 가능하도록 모양을 맞춰줌
        if stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, 
                              stride=stride, bias=False),
                nn.BatchNorm2d(out_planes)
            )
            
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

## BottelNeck Residual Block (ResNet 50, 101, 152)

In [6]:
class BottelNeck(nn.Module):
    mul = 4
    def __init__(self, in_planes, out_planes, stride=1):
        super(BottelNeck, self).__init__()
        
        # 첫 convolution 에서 너비와 높이 downsampling
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1,
                              stride=stride, bias=Flase)      
        self.bn1 = nn.BatchNorm2d(out_planes)
        
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        
        self.conv3 = nn.Conv2d(out_planes, out_planes*self.mul, 
                               kernel_size=1, stride=stride, bias=False)
        self.bn3 = nn.BatchNorm2d(out_planes*self.mul)
        
        self.shortcut = nn.Sequential()
        
        if stride != 1 or in_planes != out_planes*self.mul:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes*self.mul, 
                          kernel_size=1,stride=stride, bias=Flase),
                nn.BatchNorm2d(out_planes*self.mul)
            )
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        
        out = self.conv2(x)
        out = self.bn2(out)
        out = F.relu(out)

        out = self.conv3(x)
        out = self.bn3(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

## ResNet 구현

In [7]:
class ResNet(nn.Module):
    # CIFAR-10 를 위해 num_classes=10로 설정
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        # RGB 3개 채널에서 64개의 kernel 사용(논문 내용 참고?)
        self.in_planes = 64
        
        self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(self.in_planes)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self.make_layer(block, 64, num_block[0], stride=1)
        self.layer2 = self.make_layer(block, 128, num_block[1], stride=2)
        self.layer3 = self.make_layer(block, 256, num_block[2], stride=2)
        self.layer4 = self.make_layer(block, 512, num_block[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        
        self.linear = nn.Linear(512*block.mul, num_classes)
        
    def make_layer(self, block, out_planes, num_blocks, stride):
        stride = [stride] + [1] * (num_blocks-1)
        layers = []
        
        for i in range(num_blocks):
            layers.append(block(self.in_planes, out_planes, strides[i]))
            self.in_planes = block.mul * out_planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn1(out)
        out = F.relu(out)
        
        out = self.Maxpool1(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out        

In [8]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(BottleNeck, [3,4,6,3])

def ResNet101():
    return ResNet(BottleNeck, [3,4,23,3])

def ResNet152():
    return ResNet(BottleNeck, [3,8,36,3])